In [2]:
import pandas as pd
import string
import html
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from pandarallel import pandarallel
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import joblib
from scipy import sparse
from elasticsearch import Elasticsearch

pandarallel.initialize(nb_workers=8)

nltk.download('wordnet')


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package wordnet to /home/amogus/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
pd.read_parquet("~/resources/food/user.parquet")


NameError: name 'pd' is not defined

In [50]:
reviews_df = pd.read_parquet('resources/food/reviews.parquet')
float_cols = reviews_df.dtypes[reviews_df.dtypes != "int64"][
    reviews_df.dtypes == "float64"
].index
reviews_df.loc[:, float_cols] = reviews_df.loc[:, float_cols].fillna(0.0)
categorical_cols = reviews_df.dtypes[reviews_df.dtypes != "int64"][
    reviews_df.dtypes != "float64"
].index
reviews_df.loc[:, categorical_cols] = reviews_df.loc[:, categorical_cols].fillna("")
reviews_df


,ReviewId,RecipeId,AuthorId,AuthorName,Rating,Review,DateSubmitted,DateModified
0,2,992,2008,gayg msft,5,better than any you can get at a restaurant!,2000-01-25 21:44:00+00:00,2000-01-25 21:44:00+00:00
1,7,4384,1634,Bill Hilbrich,4,"I cut back on the mayo, and made up the differ...",2001-10-17 16:49:59+00:00,2001-10-17 16:49:59+00:00
2,9,4523,2046,Gay Gilmore ckpt,2,i think i did something wrong because i could ...,2000-02-25 09:00:00+00:00,2000-02-25 09:00:00+00:00
3,13,7435,1773,Malarkey Test,5,easily the best i have ever had. juicy flavor...,2000-03-13 21:15:00+00:00,2000-03-13 21:15:00+00:00
4,14,44,2085,Tony Small,5,An excellent dish.,2000-03-28 12:51:00+00:00,2000-03-28 12:51:00+00:00
...,...,...,...,...,...,...,...,...
1401977,2090339,139499,2002080368,terrylbiggs,2,I was disappointed. I couldn't wait to make th...,2020-12-27 23:57:54+00:00,2020-12-27 23:57:54+00:00
1401978,2090340,148484,41805321,rogerberry,5,Nothing to drain. And I don’t heat up the liqu...,2020-12-28 00:44:42+00:00,2020-12-28 00:44:42+00:00
1401979,2090341,264191,2002901848,Reiketsukan 6.,5,Good base recipe for someone to start with. I ...,2020-12-28 01:04:43+00:00,2020-12-28 01:04:43+00:00
1401980,2090345,411791,2002901938,Sue M.,5,Thank you so much for this amazing recipe! I l...,2020-12-28 03:07:10+00:00,2020-12-28 03:07:10+00:00


In [7]:
recipes_df = pd.read_parquet('resources/food/recipes.parquet')
recipes_df["RecipeId"] = recipes_df["RecipeId"].astype(int)
recipes_df


,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeYield,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,...,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,...,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,None,[Soak saffron in warm milk for 5 minutes and p...
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05 19:52:00+00:00,This is from one of my first Good House Keepi...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,None,"[Into a 1 quart Jar with tight fitting lid, pu..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03 14:54:00+00:00,This dish is best prepared a day in advance to...,[https://img.sndimg.com/food/image/upload/w_55...,...,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,4 kebabs,"[Drain the tofu, carefully squeezing out exces..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19 06:19:00+00:00,Make and share this Cabbage Soup recipe from F...,[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,None,"[Mix everything together and bring to a boil.,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522512,541379,Meg's Fresh Ginger Gingerbread,2002090414,rdsxc,PT35M,PT1H,PT1H35M,2020-12-22 15:27:00+00:00,Make and share this Meg's Fresh Ginger Gingerb...,[],...,7.6,54.4,278.2,48.5,0.8,22.8,3.9,8.0,1 8x8 cake pan,[Preheat oven to 350&deg;F Grease an 8x8 cake ...
522513,541380,Roast Prime Rib au Poivre with Mixed Peppercorns,211566,Denver cooks,PT3H,PT30M,PT3H30M,2020-12-22 15:32:00+00:00,"White, black, green, and pink peppercorns add ...",[https://img.sndimg.com/food/image/upload/w_55...,...,71.4,433.8,766.3,3.2,0.7,0.1,117.0,8.0,1 Roast,[Position rack in center of oven and preheat t...
522514,541381,Kirshwasser Ice Cream,2001131545,Jonathan F.,PT3H,PT1H,PT4H,2020-12-22 15:33:00+00:00,Make and share this Kirshwasser Ice Cream reci...,[],...,72.6,470.9,192.5,33.9,0.0,17.3,12.8,6.0,None,[heat half and half and heavy cream to a simme...
522515,541382,Quick & Easy Asian Cucumber Salmon Rolls,2001004241,CLUBFOODY,None,PT15M,PT15M,2020-12-22 22:11:00+00:00,"Extremely quick and easy to make, these are gr...",[https://img.sndimg.com/food/image/upload/w_55...,...,0.1,2.9,100.5,0.3,0.0,0.1,2.4,NaN,20 rolls,"[In a small bowl, combine mayo and wasabi past..."


In [17]:
recipe_reviews = recipes_df.set_index("RecipeId").join(reviews_df.set_index("RecipeId"), how="inner", rsuffix="_review")
recipe_reviews


,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,RecipeCategory,...,RecipeServings,RecipeYield,RecipeInstructions,ReviewId,AuthorId_review,AuthorName_review,Rating,Review,DateSubmitted,DateModified
RecipeId,,,,,,,,,,,,,,,,,,,,,
38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,...,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",579149,596506,allisonsmommy,5,"Yummy, yummy, yummy! I am a big fan of fruit a...",2008-02-13 00:26:49+00:00,2008-02-13 00:26:49+00:00
38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,...,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",789264,1060485,allyop135,4,very good!! the directions were alittle weird ...,2009-01-15 20:02:30+00:00,2009-01-15 20:02:30+00:00
38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,...,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",1448544,1802956262,ionnym,4,This does taste great! I think the recipe need...,2014-07-22 21:23:02+00:00,2014-07-22 21:23:02+00:00
38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09 21:46:00+00:00,Make and share this Low-Fat Berry Blue Frozen ...,[https://img.sndimg.com/food/image/upload/w_55...,Frozen Desserts,...,4.0,None,"[Toss 2 cups berries with sugar., Let stand fo...",1450744,2123645,DeliciousAsItLooks,4,Tasty and refreshing! I love the creamy flavor...,2014-08-14 20:51:35+00:00,2014-08-14 20:51:35+00:00
39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29 13:12:00+00:00,Make and share this Biryani recipe from Food.com.,[https://img.sndimg.com/food/image/upload/w_55...,Chicken Breast,...,6.0,None,[Soak saffron in warm milk for 5 minutes and p...,361851,393254,cassiopeia94,3,I have an Indian friend who made this dish for...,2007-01-20 14:00:09+00:00,2007-01-20 14:00:09+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540899,Butter Pecan Banana Muffins,1827868,jarp4,PT18M,PT10M,PT28M,2020-09-01 20:59:00+00:00,Make and share this Butter Pecan Banana Muffin...,[],Breads,...,12.0,12 Muffins,[Heat oven to 375 degrees. Coat a standard 12-...,2087065,2002811014,Rick J.,5,We had these for breakfast this morning. They ...,2020-09-03 17:55:43+00:00,2020-09-03 17:55:43+00:00
541001,Homemade Adobo Seasoning,181781,Wing-Man,PT1M,PT15M,PT16M,2020-09-24 16:53:00+00:00,Make and share this Homemade Adobo Seasoning r...,[],Mexican,...,50.0,1 Pint,"[Mix all ingredients together., Store in a tig...",2087640,2000325639,robert p.,0,"This appears to be a dry Mexican style adobo, ...",2020-09-25 21:07:47+00:00,2020-09-25 21:07:47+00:00
541030,Everything but the Bagel Seasoning (Trader Joe...,50509,duonyte,None,PT5M,PT5M,2020-10-02 20:48:00+00:00,This is a copycat of this popular seasoning mi...,[],< 15 Mins,...,NaN,3/4 cup,[Mix everything together and transfer to an ai...,2088114,266635,DailyInspiration,5,Made this mix to go with bagels and cream chee...,2020-10-16 23:31:55+00:00,2020-10-16 23:31:55+00:00


### password: 
ABPXxOfVz8=_ehPbyN0=
### kibana:
eyJ2ZXIiOiI4LjEyLjIiLCJhZHIiOlsiMTcyLjE4LjAuMjo5MjAwIl0sImZnciI6ImIwMWI5ZGU5ZGM0NTU5MzNlZGM1NzliODk3NWZkMDdlNWZkMjUwZDQ2ZjU3MGRlZTA5MjkyMWM5ZDMxM2FkN2EiLCJrZXkiOiJqSTNzTVk0QmRBTUNjS2hxdE5YSzo3TGhrR3Y4NlFrT1FERkExMkQ3VzFnIn0=

In [46]:
from elasticsearch import Elasticsearch, helpers

es = Elasticsearch(
    "https://127.0.0.1:9200",
    basic_auth=(
        "elastic",
        "yHcm1Pyq=jnDL_4gw93i",
    ),
    ca_certs="http_ca.crt"
)
es.info()


ObjectApiResponse({'name': '9429c74bab12', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'PzQzy0-rTiKgmDWiUpey6A', 'version': {'number': '8.12.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '48a287ab9497e852de30327444b0809e55d46466', 'build_date': '2024-02-19T10:04:32.774273190Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [58]:
def generate_indices():
    for index, row in reviews_df.iterrows():
        yield {"_index": f"user_{row.AuthorId}", "_id": row.RecipeId, **row.to_dict()}

helpers.bulk(es, generate_indices(), ignore_status=[400, 404])


(17002,
 [{'index': {'_index': 'user_2416',
    '_id': '1451',
    'status': 400,
    'error': {'type': 'validation_exception',
     'reason': 'Validation Failed: 1: this action would add [2] shards, but this cluster currently has [999]/[1000] maximum normal shards open;'}}},
  {'index': {'_index': 'user_2536',
    '_id': '3338',
    'status': 400,
    'error': {'type': 'validation_exception',
     'reason': 'Validation Failed: 1: this action would add [2] shards, but this cluster currently has [999]/[1000] maximum normal shards open;'}}},
  {'index': {'_index': 'user_2639',
    '_id': '1078',
    'status': 400,
    'error': {'type': 'validation_exception',
     'reason': 'Validation Failed: 1: this action would add [2] shards, but this cluster currently has [999]/[1000] maximum normal shards open;'}}},
  {'index': {'_index': 'user_2509',
    '_id': '8476',
    'status': 400,
    'error': {'type': 'validation_exception',
     'reason': 'Validation Failed: 1: this action would add [2] s